In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Build a model that can classify if there is a bird in an image
Steps
1. Collect dataset from internet - using ddg search
2. download images
3. check the data and clean the invalid images
4. create a dataloader
5. create a model
6. train the model using resnet or other arch (first train a simple model)
7. check the images with largest loss and clean the bad input data
8. train the model again
9. test the model
10. export the model

In [ ]:
# installs
!pip install -U duckduckgo_search


In [ ]:
# imports
from duckduckgo_search import DDGS
from fastcore.all import *
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
from fastdownload import download_url
import time

In [ ]:
# func to search images
def search_images_ddg(topic, max=100):
    urls = L(DDGS().images(keywords=topic, max_results=max)).itemgot('image')
    return urls

# label function
def get_y(item):
    return 0

def downloadImages(urls, path):
    for url in urls:
        download_url(url, path, show_progress=False)
    print(f"Images downloaded into {path}")

In [ ]:
topics = "birds", "forest"
path = Path('bird_or_not')

for topic in topics:
    dest = path/topic
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images_ddg(topic))
    time.sleep(5)
    resize_images(dest, max_seiz=400, dest=dest)

In [ ]:
# remove invalid images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
# define dataloader
dls = DataBlock(blocks=(ImageBlock, CategoryBlock),
                get_items=get_image_files,
                splitter=RandomSplitter(seed=32),
                get_y=parent_label,
                item_tfms=[Resize(192, method='squish')]
               ).dataloaders(path, bs=32)


In [ ]:
dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet50, metrics=error_rate)

In [ ]:
# train over one cycle
learn.fine_tune(3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
# check and clean the data
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx, cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
(path/'birds').ls()

In [ ]:
image = path/'birds'/'2fcb6b6a-06c2-408d-b2b1-62378087ea3e.jpg'
learn.predict(image)

In [ ]:
modelFile = 'isBirdModelResnet50.pkl'
learn.export(modelFile)

In [ ]:
learn_loaded = load_learner(modelFile)

In [ ]:
learn_loaded.predict(image)